# Web applications with Python - Bottle
## Introduction to web applications

Web applications or websites can be created in multiple ways. Both when it comes to the engine/software (PHP, Python, Ruby etc.) and architecture of the application. In practice, Model View Controller (MVC) or Model View Presenter (MVP, sometimes called a successor to MVC) are the most common architectures. You should know how MVC looks like and create applications according to this rule.

Model View Controller:
* Model - responsible for data structure. Model represents knowledge/data. A single table or a complex database may be a model.
* View - View is the way of displaying a model. View is responsible for presenting data to the user.
* Controller - engine which decides what to do with the model (query the model, modify the model, display view with model's data)

Thorough understanding of the philosophy behind each architecture is mostly unnecessary for beginners. However, you should remember that data storage (model), actions connected with data and reacting to user's input (controller) and way of displaying data (view) are three separate layers which should not be combined. It is not particularly important if you choose MVC or MVP.

Read more here:
* https://en.wikipedia.org/wiki/Model%E2%80%93view%E2%80%93controller
* https://blog.codinghorror.com/understanding-model-view-controller/
* https://www.codeproject.com/Articles/288928/Differences-between-MVC-and-MVP-for-Beginners

## Webservers, frameworks, template languages etc.
One of the more common solutions is LAMP - Linux, Apache, MySQL, and PHP. In Linux operating system, webserver communicates with the net, PHP prepares the content from the MySQL database. In practice solutions/libraries/frameworks make our life easier, so that we do not need to write everything from scratch.

In case of Python, Web Server Gateway Interface (WSGI) is usually used. It allows you to combine Python applications with webserver capabilities (communication with web browsers). Fortunately you do not have to understand every part of the webserver when using any Python framework. For a beginner, it "just works".

If yoy have a working Python installation with a framework you can publish your application immediately.

See also:
* https://www.fullstackpython.com/wsgi-servers.html

## Python frameworks
There are many popular Python frameworks. Most commonly used are probably Bottle, Flask, and Django. Bottle, which will be a starting point for you is the smallest and easiest solution, perfectly suited for beginners. Flask is a more advanced alternative, and Django truly is a Swiss knife of web frameworks. Django is used by large websites (Instagram, Washington Post, Pinterest etc.). To sum up, Bottle is a good first step in creating web applications and allows us access to an endless world.

## Bottle
Bottle is a small and convenient framework, in which you can create interesting applications fast. The most distinctive feature of Bottle is that it requires you to write an application in one file. Of course you can use imports, but still it is a limitation which makes Bottle unsuitable for large projects. In a moment you will see how much you can achieve with a simple script.

* https://bottlepy.org/docs/dev/

### The simplest application
The simplest application looks like this:
```python
from bottle import route, run, template

@route('/hello/<name>')
def index(name):
    return template('<b>Hello {{name}}</b>!', name=name)

run(host='localhost', port=8080)
```

In this case you may see that the Python script imports parts of Bottle and defines a route - address within the server - with assigned Python function. Funtion returns a template (view), which is a simple, one-line HTML in this case.

Note that to create web applications in Python you still need the basics of HTML, CSS and JavaScript. Fortunately you will gain this knowledge in practice, just copying and modifying ready-made solutions at the beginning.

If you want to learn HTML and CSS it is a good idea to visit online tutorials, for example:
* https://learn.shayhowe.com/html-css/
* https://learn.shayhowe.com/advanced-html-css/

## Extending an application

See how an application with two subpages looks like:
```python
from bottle import route, run, template

@route('/hello/<name>')
def hello(name):
    return template('<b>Hello {{name}}</b>!', name=name)

@route('/bye/<name>')
def bye(name):
    return template('<b>Bye, Bye {{name}}</b>!', name=name)

run(host='localhost', port=8080)
```

As you can see, it was enough to add a few lines. Now let us describe the elements.

```python
@route('/hello/<name>')
```
Tells Bottle that a function is defined there. The function returns a template when you access the page: [our site]/hello, and name is an argument.

If you access the site:

[our site]/hello/Maciej

You will see:

"Hello Maciej"

What you can see as the result of the function? It tells us that to a `<b>Hello {{name}}</b>!` view you should pass a "name" variable as "name", so that the view knows what to put in there.

```python
    return template('<b>Hello {{name}}</b>!', name=name)
```


## Basic application
You may see most important parts of the application in an example in the catalog 08_Webapp. Let us start analyzing the server.py file, beginning with the end.

```python
@app.route('/')
@app.route('/index')
@app.route('/index/')
@app.route('/index/<message>')
def index(message=''):
    loginName = checkAuth()
    messDict = {'error': "Something went wrong",
                'ok': "Everything is ok."}
    return template('index', message=messDict.get(message, ""), loginName=loginName)
```

You define an index function for a few names. This function calls checkAuth() (look below). If everything went fine, it calls a template 'index' and passes two variables to it, message and loginName.

Note that template is a simple string. If it is the case, Bottle looks for a file in templates catalog. Look at the contents:

```html
% rebase('base.tpl', title='Python')
<div class="row">
	<div class="col-md-12">
	<h3>{{message}}</h3>
	Hello {{loginName}}
	</div>
</div>
```

Contents of index.html are self-explanatory. You may expect that contents of variables will be inserted into {{message}} and {{loginName}}.

% rebase('base.tpl', title='Python') makes contents of the template being inserted into base.tpl, so that you may keep common parts of the application in this template (e.g. navbar).

### Authorization
Look what checkAuth() does.

```python
def checkAuth():
    loginName = request.get_cookie("user", secret=secretKey)
    randStr = request.get_cookie("randStr", secret=secretKey)
    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))
    if (loginName in users) and (users[loginName].get("randStr", "") == randStr) and (users[loginName]["loggedIn"] == True) and (time.time() - users[loginName]["lastSeen"] < 3600):
        users[loginName]["lastSeen"] = time.time()
        return loginName
    return redirect('/login')
```


* At the beginning, try to access an encrypted cookie file for a given user.
* Log in a file that a user tried to do something at our website.
* Check if they are actually logged in.
* If yes, update their activity time and return loginName.
* If no, redirect them to login page.


### Logging in
Logging in part should be rather easy.

```python
@app.route('/login')
@app.route('/login/')
@app.route('/login', method='POST')
def login():
    loginName = request.forms.get('login_name', default=False)
    password = request.forms.get('password', default=False)
    randStr = ''.join(random.choice(
        string.ascii_uppercase + string.digits) for _ in range(18))
    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))
    if (loginName in users) and users[loginName]["password"] == password:
        response.set_cookie("user", loginName, secret=secretKey)
        response.set_cookie("randStr", randStr, secret=secretKey)
        users[loginName]["loggedIn"] = True
        users[loginName]["randStr"] = randStr
        users[loginName]["lastSeen"] = time.time()
        
        redirect('/index')
        return True
    else:
        return template('login')
    return template('login')
```

* @app.route('/login', method='POST') tells a server that POST data will be passed (data from a form).
* Try to access data from a form: loginName = request.forms.get('login_name', default=False)
* Generate a random string for a user. It makes us safe from forged cookies or using outdated cookies.
* Log the activity in a log file.
* Check if the password is correct. You must never store passwords in plain text, always use hashes in production, see https://crackstation.net/hashing-security.htm.

### Other
Other parts are simple and do not require changes.
```python
@app.route('/static/:path#.+#', name='static')
def static(path):
    return static_file(path, root='./static')
```
Tell Bottle that some elements are static and they are not a function or Python code.

```python
app = Bottle()
```
Create an instance of the application.

```python
from users import users
```
Read user data from a dictionary (it should eventually be a database). The dictionary of uses allows you to store data in a session.

```python
secretKey = "SDMDSIUDSFYODS&TTFS987f9ds7f8sd6DFOUFYWE&FY"
```
Define a secret key. This key should be hidden, because it is used to encrypt cookies. The key should be unique (different) for every application.

```python
log = logging.getLogger('bottle')
log.setLevel('INFO')
h = logging.handlers.TimedRotatingFileHandler(
    'logs/nlog', when='midnight', backupCount=9999)
f = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
h.setFormatter(f)
log.addHandler(h)
```
Define how logging in works.
Read more about logging in:
* https://docs.python.org/3/library/logging.html
* https://fangpenlin.com/posts/2012/08/26/good-logging-practice-in-python/

## Streamlit

A simple library that can be used for rapid prototyping or building quick web apps in python is called Streamlit.

You can read more about it as see examples of the capabilities on their website: https://streamlit.io/

Below is a simple example of a web app built with streamlit, using just a few lines of code.

Copy it into a .py file and then run using `streamlit run streamlit_app.py`

```python

# streamlit_app.py

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


st.title("Streamlit Demo Application")
st.write("This is a simple demonstration of Streamlit capabilities.")

# Sidebar
st.sidebar.header("User Input")
user_name = st.sidebar.text_input("Enter your name:", "Guest")
data_points = st.sidebar.slider("Number of data points:", min_value=10, max_value=500, value=100)
chart_type = st.sidebar.selectbox("Choose chart type:", ["Line", "Bar", "Area"])


st.write(f"Hello, **{user_name}!** Let's explore some data!")

# Generate random dataset
data = pd.DataFrame({
    "X": np.arange(1, data_points + 1),
    "Y": np.random.randn(data_points).cumsum()
})

# Display data
st.subheader("Generated Dataset")
st.dataframe(data)

# Descriptive stats
st.subheader("Basic Data Statistics")
st.write(data.describe())

# Visualization
st.subheader("Data Visualization")
if chart_type == "Line":
    st.line_chart(data.set_index("X"))
elif chart_type == "Bar":
    st.bar_chart(data.set_index("X"))
elif chart_type == "Area":
    st.area_chart(data.set_index("X"))

# Matplotlib plot
st.subheader("Matplotlib Plot")
fig, ax = plt.subplots()
ax.plot(data["X"], data["Y"], label="Cumulative Sum", color="blue", alpha=0.7)
ax.set_title("Matplotlib Plot")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.legend()
st.pyplot(fig)

# Footer
st.write("Thank you for using this demo! 🎉")

```